# My first CTMC algorithm

My first attempt at fitting a Continous Time Markov Chain to simulated data in a toy problem. Mathematical background on how to build the likelihood can be found in [here](http://www.mathematik.uni-karlsruhe.de/ianm3/~jahnke/media/medijasc2007.pdf).

### Simulate toy problem

First we will simulate some data. We will use a a simple 3 state model, a typical SIRS epidemic model with perfect vaccination, on a closed population of 100 people.

The transitions will be defined as follows:

\begin{align}
S \to I & &\text{with rate} & &\beta \frac{SI}{N},\\
I \to R & &\text{with rate} & &\gamma \frac{I}{N},\\
R \to S & &\text{with rate} & &\omega \frac{R}{N},\\
S \to R & &\text{with rate} & &\nu \frac{I}{N}.\\
\end{align}

For simplicity, we can define the set of parameters to be $\boldsymbol{\theta} = (\beta, \gamma, \omega, \nu)$. For our toy problem, let us set $\boldsymbol{\theta} = ( 2, 0.125, 1/400, 10)$, with an initial state (starting population) completely susceptible with 1 infected seed, i.e. $\boldsymbol{n} = (99,1, 0)$

In [65]:
import numpy as np

init = np.array((99,1,0))
beta = 2
gamma = 0.125
omega = 1/400
nu = 10
theta = np.array((beta, gamma,omega,nu))

class sirs:
    
    """Class object to contain the ctmc model"""
    def __init__(self,beta, gamma, omega, nu, init):
        self.parameters = (beta, gamma, omega,nu)
        assert len(init)==3, "Number of states given is not 3"
        self.initial = init
        self.current = list(init)
        self.totalpop = sum(init)
    
    def transition(self):
        
        
        #First define transition movements
        
        def infection():
            #Infected transmission
            self.current[0] -= 1
            self.current[1] += 1

        def recovery():
            #Recovery 
            self.current[1] -=1
            self.current[2] +=1


        def vacc():
            self.current[0] -=1
            self.current[2] +=1

        def waning():
            self.current[2] -=1
            self.current[0] +=1

        map_transitions = {
            0 : infection,
            1 : recovery,
            2 : vacc,
            3 : waning,
        }

        #Calculate transitions rates
        trans = beta * self.current[0]*self.current[1]/self.totalpop
        recov = gamma * self.current[1]
        wane = omega * self.current[2]
        vac = nu * self.current[1]/self.totalpop
        total = trans + recov + wane + vac
        
        #Record current rates
        self.currentrates = (trans, recov, wane, vac)
        
        #Draw a transition time
        dt = np.random.exponential(total)
        
        #Draw a transition
        p = np.array(self.currentrates)/total
        s = np.random.choice(4, p=p)
        map_transitions[s]()
        
        return dt, s
        

In [66]:
toy = sirs(beta, gamma, omega, nu, init)
toy.transition()

0
[98, 2, 0]


In [67]:
init = (10,30,50)
toy = sirs(beta, gamma, omega, nu, init)
toy.transition()

3
[11, 30, 49]


In [40]:
toy.currentrates

(6.666666666666667, 3.75, 0.125, 3.3333333333333335)